In [1]:
import sys
import numpy as np
import MDAnalysis as mda
import mbuild as mb
import textwrap
# import inhouse functions
import systop_utils

ModuleNotFoundError: No module named 'systop_utils'

In [2]:
path = '/Users/joedinski/work/Projects/FAIRmat/nomad/dependencies/parsers/lammps/tests/data/1_methyl_naphthalene/'
sys_nm = '1_methyl_naphthalene'
universe = mda.Universe(path+'data.'+sys_nm, path+'naph_298_eq.lammpstrj', topology_format='DATA', format='LAMMPSDUMP')

In [3]:
# Construct System Topology
# mol / fragments
# segments
# residues / monomers
# atoms
system = systop_utils.Compound() # store the topology in the mbuild class "Compound"

In [4]:
# first get the atom attributes from the MDAnalysis universe
# nb - moltypes, molnums, elements, and names are defined according to the information available
# moltypes <--> fragtypes
# molnums <--> fragnums
# elements <--> types
# names <--> types
system.get_atom_attributes(universe)
print(system.atoms.moltypes)
print(system.atoms.molnums)
print(system.atoms.resnames)
print(system.atoms.resids)
print(system.atoms.elements)
print(system.atoms.names)

['0' '0' '0' ... '0' '0' '0']
[ 0  0  0 ... 53 53 53]
['1' '1' '1' ... '1' '1' '1']
[1 1 1 ... 1 1 1]
['2' '2' '2' ... '5' '4' '4']
['2' '2' '2' ... '5' '4' '4']


In [5]:
# use the atom attributes to construct the basic hierarchical structure molecules-->segments-->residues-->atoms
system.construct_topology()

In [6]:
# add the bonds to the topology structure using the MDAnalysis universe
# nb - the mbuild Compound class stores the bonds as a graph (similar to a networkx object)
#    - it is quite expensive to add the bonds one by one...not even sure what the utility of this is, we might just add
#       a reference to the bond list in the force field (or vica versa).
system.add_bonds_from_universe(universe)

In [7]:
# we can add a representative structure to the topology structure
# nb - this could just be a reference to the input/starting configuration
system.add_positions_from_universe(universe)

In [8]:
# Let's illustrate the topology
print('The system is '+str(system))
print('The first molecule is '+str(system.children[0]))
#print('The first residue of the first molecule is '+str(system.children[0].children[0]))
#print('The first atom in the first residue of the first molecule is '+str(system.children[0].children[0].children[0]))
print('The first atom in the first molecule is '+str(system.children[0].children[0]))

The system is <Compound 1134 particles, non-periodic, 1188 bonds, id: 140494741457616>
The first molecule is <Compound 21 particles, non-periodic, 22 bonds, id: 140494741436304>
The first atom in the first molecule is <2 pos=(-9.1562,-9.8219,-10.3730), 0 bonds, id: 140494741436688>


In [9]:
# with the mbuild Compound class, we can easily view the system or subsystems with nglview
system.children[0].children[0]._visualize_nglview()

/Users/joedinski/miniconda3/envs/mosdef37/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<UNK pos=(-9.1562,-9.8219,-10.3730), 0 bonds, id: 140494741436688>" is element: "U"
  atom, element))
/Users/joedinski/miniconda3/envs/mosdef37/lib/python3.7/site-packages/mbuild/box.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mins = np.array(mins, dtype=np.float)
/Users/joedinski/miniconda3/envs/mosdef37/lib/python3.7/site-packages/mbuild/box.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you

NGLWidget()

In [10]:
# Here we use the atom attributes to gather a summary/overview of the topology
# nb - this could probably also be done directly from the topology data structure...that would probably be more general/robust
system.get_topology_summary()

In [11]:
for moltype in system.molnames:
    print('MOLTYPE = '+moltype+':')
    print('Number of molecules of this type: '+str(system.moltypes[moltype].count))
    if system.moltypes[moltype].n_res == 1:
        print('A dictionary with the counts of each atom element/type for this molecule type: '+str(system.moltypes[moltype].atom_count))
        print('The formula of this molecule type, with respect to the atoms: '+str(system.moltypes[moltype].atom_formula))
    else:
        print('A dictionary with the counts of each residue type within this molecule type: '+str(system.moltypes[moltype].residue_count))
        print('The formula of this molecule type, with respect to residues: '+str(system.moltypes[moltype].residue_formula))
        print('The full sequence of residues for this molecule type: '+str(system.moltypes[moltype].residue_sequence))
        for res in system.moltypes[moltype].resnames:
            print('A dictionary with the counts of each atom element/type for this residue type (and molecule type): '+str(system.moltypes[moltype].restypes[res].atom_count))
            print('The formula of each residue type (for this molecule type), with respect to the atoms: '+str(system.moltypes[moltype].restypes[res].atom_formula))

MOLTYPE = 0:
Number of molecules of this type: 54


AttributeError: 'moltypes' object has no attribute 'residue_count'

In [34]:
from mbuild.formats.json_formats import compound_to_json, compound_from_json
compound_to_json(system,'test.json')

TypeError: Object of type float32 is not JSON serializable

In [ ]:
system